# European electricity price EDA

This notebook inspects the wholesale electricity price dataset under `data/raw/electricity_price/european_wholesale_electricity_price_data_monthly.csv`. The goal is to understand coverage, detect missing values, and highlight countries or periods with notable price shifts.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

plt.style.use("seaborn-v0_8-colorblind")


def find_project_root(start: Path, marker: str = "data") -> Path:
    """Walk up directories until the given marker folder is found."""
    current = start.resolve()
    while True:
        if (current / marker).exists():
            return current
        if current.parent == current:
            raise FileNotFoundError(f"Could not locate '{marker}' directory")
        current = current.parent


PROJECT_ROOT = find_project_root(Path.cwd())
DATA_PATH = (
    PROJECT_ROOT
    / "data/raw/electricity_price/european_wholesale_electricity_price_data_monthly.csv"
)
print(f"Using data file: {DATA_PATH}")

df = (
    pd.read_csv(DATA_PATH, parse_dates=["Date"])
    .sort_values("Date")
    .reset_index(drop=True)
)
df.head()

## 1. Dataset overview & completeness

In [ ]:
n_obs = len(df)
n_countries = df["Country"].nunique()
start_date = df["Date"].min()
end_date = df["Date"].max()
print(f"Observations: {n_obs:,}")
print(f"Countries: {n_countries}")
print(f"Date range: {start_date.date()} to {end_date.date()}")
df.describe(include="all")

In [ ]:
missing = (
    df.isna()
    .sum()
    .to_frame(name="missing_count")
    .assign(missing_pct=lambda d: (d["missing_count"] / len(df)) * 100)
)
missing

### Rows with missing values

In [ ]:
missing_rows = df[df.isna().any(axis=1)].copy()
print(f"Rows with NaNs: {len(missing_rows)}")
missing_rows

## 2. Temporal structure & frequency

In [ ]:
unique_times = df["Date"].dt.time.unique()
print(f"Unique time components: {len(unique_times)}")
print(sorted(unique_times)[:3])

day_values = df["Date"].dt.day.unique()
print(f"Unique day-of-month values: {sorted(day_values)}")

delta_days = (
    df.sort_values(["Country", "Date"])
    .groupby("Country")["Date"]
    .diff()
    .dt.days.dropna()
)
if not delta_days.empty:
    print("Distribution of gaps between successive records (days):")
    print(delta_days.describe())
    print("Most common gaps (days):")
    print(delta_days.value_counts().head())

monthly_check = (
    df.assign(month=df["Date"].dt.to_period("M")).groupby(["Country", "month"]).size()
)
print("Max obs per country-month:", monthly_check.max())
if monthly_check.max() == 1 and set(day_values) == {1}:
    print("Dataset is monthly.")

## 3. Coverage and onboarding

How many countries report prices each month, and when do new markets enter the dataset?

### 3.1 Countries reporting per month

In [ ]:
countries_per_month = (
    df.groupby("Date")["Country"]
    .nunique()
    .rename("countries_reporting")
    .reset_index()
    .sort_values("Date")
)
print(f"Total months covered: {len(countries_per_month)}")
print(
    "Min/Max number of countries per month:",
    countries_per_month["countries_reporting"].min(),
    countries_per_month["countries_reporting"].max(),
)
countries_per_month

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(
    countries_per_month["Date"], countries_per_month["countries_reporting"], marker="o"
)
ax.set_ylabel("Number of countries reporting")
ax.set_xlabel("Month")
ax.set_title("Coverage by month (number of countries with prices)")
plt.tight_layout()

### 3.2 First appearance by country

In [ ]:
country_onboarding = (
    df.groupby("Country")["Date"]
    .min()
    .reset_index(name="first_month")
    .sort_values("first_month")
)
country_onboarding["first_month"] = (
    country_onboarding["first_month"].dt.to_period("M").dt.to_timestamp()
)
print(f"Countries tracked: {len(country_onboarding)}")
country_onboarding

### 3.3 Onboarding pace

In [ ]:
monthly_additions = (
    country_onboarding.groupby("first_month")["Country"]
    .agg(countries_added="count", countries=list)
    .reset_index()
)
monthly_additions["cumulative"] = monthly_additions["countries_added"].cumsum()

fig, ax = plt.subplots(figsize=(12, 4))
ax.bar(
    monthly_additions["first_month"],
    monthly_additions["countries_added"],
    width=20,
    alpha=0.6,
    label="Countries added in month",
)
ax2 = ax.twinx()
ax2.plot(
    monthly_additions["first_month"],
    monthly_additions["cumulative"],
    color="tab:red",
    label="Cumulative countries",
)
ax.set_ylabel("Countries added")
ax2.set_ylabel("Cumulative total")
ax.set_title("Country additions over time")
ax.legend(loc="upper left")
ax2.legend(loc="upper right")
plt.tight_layout()

### 3.4 Annotated coverage timeline

In [ ]:
first_data_month = country_onboarding["first_month"].min()
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(
    countries_per_month["Date"],
    countries_per_month["countries_reporting"],
    marker="o",
    label="Countries reporting",
)
ax.set_ylabel("Number of countries reporting")
ax.set_xlabel("Month")
ax.set_title("Coverage with country arrivals annotated")
ax.legend(loc="upper left")

for idx, row in monthly_additions.iterrows():
    if row["first_month"] == first_data_month:
        continue
    label_list = row["countries"]
    if not label_list:
        continue
    label = ", ".join(label_list)
    if len(label) > 45:
        label = label[:42] + "..."
    mask = countries_per_month["Date"] == row["first_month"]
    if not mask.any():
        continue
    y_val = countries_per_month.loc[mask, "countries_reporting"].iloc[0]
    offset = 25 if idx % 2 == 0 else -35
    va = "bottom" if offset > 0 else "top"
    ax.annotate(
        label,
        xy=(row["first_month"], y_val),
        xytext=(0, offset),
        textcoords="offset points",
        ha="center",
        va=va,
        arrowprops=dict(arrowstyle="->", color="gray", lw=1),
        fontsize=9,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.8),
    )

plt.tight_layout()

## 4. Seasonal patterns

In [ ]:
month_stats = (
    df.assign(month=df["Date"].dt.month, month_name=df["Date"].dt.month_name())
    .groupby(["month", "month_name"])["Price (EUR/MWhe)"]
    .agg(mean="mean", median="median", std="std")
    .reset_index()
    .sort_values("month")
)
month_stats

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(month_stats["month_name"], month_stats["mean"], marker="o", label="Mean price")
ax.fill_between(
    month_stats["month_name"],
    month_stats["mean"] - month_stats["std"],
    month_stats["mean"] + month_stats["std"],
    color="#1f77b4",
    alpha=0.15,
    label="±1 std dev",
)
ax.set_ylabel("EUR/MWhe")
ax.set_xlabel("Month of year")
ax.set_title("Average wholesale price by month across years")
ax.legend(loc="upper right")
ax.tick_params(axis="x", rotation=45)
plt.tight_layout()

### 4.1 Seasonal profile excluding crisis years
The 2021-2023 crisis inflates the monthly averages. Below we recompute seasonality after removing those months.

In [ ]:
crisis_mask = (df["Date"] >= "2021-07-01") & (df["Date"] <= "2023-05-01")
seasonal_no_crisis = (
    df.loc[~crisis_mask]
    .assign(month=df["Date"].dt.month, month_name=df["Date"].dt.month_name())
    .groupby(["month", "month_name"])["Price (EUR/MWhe)"]
    .agg(mean="mean", median="median", std="std")
    .reset_index()
    .sort_values("month")
)
print(f"Observations kept: {len(df.loc[~crisis_mask])} (out of {len(df)})")
seasonal_no_crisis

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(
    seasonal_no_crisis["month_name"],
    seasonal_no_crisis["mean"],
    marker="o",
    label="Mean (ex-crisis)",
)
ax.fill_between(
    seasonal_no_crisis["month_name"],
    seasonal_no_crisis["mean"] - seasonal_no_crisis["std"],
    seasonal_no_crisis["mean"] + seasonal_no_crisis["std"],
    color="#2ca02c",
    alpha=0.15,
    label="±1 std dev",
)
ax.set_ylabel("EUR/MWhe")
ax.set_xlabel("Month of year")
ax.set_title("Average wholesale price by month (excluding crisis)")
ax.legend(loc="upper right")
ax.tick_params(axis="x", rotation=45)
plt.tight_layout()

## 5. Monthly price dynamics

In [ ]:
monthly_stats = (
    df.groupby("Date")["Price (EUR/MWhe)"]
    .agg(mean="mean", median="median", min="min", max="max", std="std")
    .reset_index()
)
monthly_stats.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(monthly_stats["Date"], monthly_stats["mean"], label="Mean price")
ax.fill_between(
    monthly_stats["Date"],
    monthly_stats["min"],
    monthly_stats["max"],
    color="#1f77b4",
    alpha=0.15,
    label="Min/Max range",
)
ax.set_ylabel("EUR/MWhe")
ax.set_title("Monthly wholesale price trend")
ax.legend(loc="upper left")
plt.tight_layout()

### 5.2 Monthly mean table

In [ ]:
monthly_trend_table = monthly_stats[["Date", "mean"]].copy()
monthly_trend_table

### 5.1 Countries driving the min/max envelope

In [ ]:
idx_min = df.groupby("Date")["Price (EUR/MWhe)"].idxmin()
idx_max = df.groupby("Date")["Price (EUR/MWhe)"].idxmax()
monthly_min = df.loc[idx_min, ["Date", "Country", "Price (EUR/MWhe)"]].rename(
    columns={"Country": "min_country", "Price (EUR/MWhe)": "min_price"}
)
monthly_max = df.loc[idx_max, ["Date", "Country", "Price (EUR/MWhe)"]].rename(
    columns={"Country": "max_country", "Price (EUR/MWhe)": "max_price"}
)
monthly_extremes = monthly_min.merge(monthly_max, on="Date")

mask = (monthly_extremes["Date"] >= "2018-01-01") & (
    monthly_extremes["Date"] <= "2021-12-01"
)
print("Most common min contributors between 2018 and 2021:")
print(monthly_extremes.loc[mask, "min_country"].value_counts().head())
print("Example rows:")
print(monthly_extremes.loc[mask, ["Date", "min_country", "min_price"]].head())
monthly_extremes

## 6. Distribution of monthly prices

In [ ]:
price_summary = df["Price (EUR/MWhe)"].describe(
    percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]
)
price_summary

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(df["Price (EUR/MWhe)"], bins=40, color="#1f77b4", alpha=0.8)
ax.set_xlabel("Monthly wholesale price (EUR/MWhe)")
ax.set_ylabel("Number of records")
ax.set_title("Distribution of price observations")
plt.tight_layout()

## 7. Yearly averages

In [ ]:
yearly_stats = (
    df.assign(Year=df["Date"].dt.year)
    .groupby("Year")["Price (EUR/MWhe)"]
    .agg(mean="mean", median="median", min="min", max="max")
    .reset_index()
)
yearly_stats

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(yearly_stats["Year"], yearly_stats["mean"], marker="o")
ax.set_title("Average annual wholesale price")
ax.set_ylabel("EUR/MWhe")
ax.set_xlabel("Year")
plt.tight_layout()

## 8. Country-level differences

In [ ]:
country_stats = (
    df.groupby("Country")["Price (EUR/MWhe)"]
    .agg(
        mean="mean",
        median="median",
        min="min",
        max="max",
        std="std",
        observations="count",
    )
    .sort_values("mean", ascending=False)
)
country_stats.head(10)

In [ ]:
country_stats.tail(10)

### 8.1 Monthly volatility by country

In [ ]:
df_country = df.sort_values(["Country", "Date"]).copy()
df_country["monthly_pct_change"] = df_country.groupby("Country")[
    "Price (EUR/MWhe)"
].pct_change(fill_method=None)
volatility = (
    df_country.groupby("Country")["monthly_pct_change"]
    .agg(mean_change="mean", change_std="std", observations="count")
    .dropna()
    .sort_values("change_std", ascending=False)
)
volatility.head(10)

## 9. Country/ISO alignment

In [ ]:
unique_pairs = df[["Country", "ISO3 Code"]].drop_duplicates().reset_index(drop=True)
print(f"Unique (Country, ISO3) pairs: {len(unique_pairs)}")
country_iso = (
    unique_pairs.groupby("Country")["ISO3 Code"].nunique().reset_index(name="iso_codes")
)
iso_country = (
    unique_pairs.groupby("ISO3 Code")["Country"]
    .nunique()
    .reset_index(name="country_names")
)

mismatched_country = country_iso[country_iso["iso_codes"] > 1]
mismatched_iso = iso_country[iso_country["country_names"] > 1]

if mismatched_country.empty and mismatched_iso.empty:
    print("Every country maps to a single ISO3 code and vice versa.")
else:
    if not mismatched_country.empty:
        print("Countries mapped to multiple ISO codes:")
        display(mismatched_country)
    if not mismatched_iso.empty:
        print("ISO codes mapped to multiple countries:")
        display(mismatched_iso)

## 10. Most recent month snapshot

In [ ]:
latest_date = df["Date"].max()
latest_month = df[df["Date"] == latest_date].sort_values(
    "Price (EUR/MWhe)", ascending=False
)
print(f"Latest month in data: {latest_date.date()}")
print("Top 5 most expensive markets")
print(latest_month.head(5))
print("Top 5 cheapest markets")
print(latest_month.tail(5))

## 11. Takeaways

* Coverage spans 2015-01-01 through 2025-08-01 with 31 countries represented.
* The highest annual mean price appears in 2022 (~223.4 EUR/MWhe).
* Highest priced markets on average: Ireland, Montenegro, North Macedonia, Croatia, Serbia.
* Most affordable markets on average: Estonia, Denmark, Finland, Norway, Sweden.
* Highest price volatility (stdev of monthly % change): Norway, Finland, Sweden.


# Electricity price seasonality analysis

This notebook focuses on the time-series nature of the European wholesale electricity prices. We build a single European benchmark series (mean of all countries per month) and inspect its trend, seasonal pattern, and correlation structure (ACF/PACF).

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

raw = pd.read_csv(DATA_PATH, parse_dates=["Date"]).sort_values("Date")
raw.head()

## Build aggregate benchmark series

In [ ]:
series = raw.groupby("Date")["Price (EUR/MWhe)"].mean().sort_index()
series = series.asfreq("MS")
series = series.interpolate("time")
series.name = "Avg price (EUR/MWhe)"

print(series.head())
print(series.tail())
print(
    f"Time span: {series.index.min().date()} -> "
    f"{series.index.max().date()} ({len(series)} months)"
)

## Overall monthly trend

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(series.index, series, label="Monthly average price")
ax.set_ylabel("EUR/MWhe")
ax.set_title("European wholesale electricity price (mean across countries)")
ax.legend()
plt.tight_layout()

## Seasonal decomposition
We decompose the series into trend, seasonal, and residual components (additive model, period=12).

In [ ]:
decomp = seasonal_decompose(
    series, model="additive", period=12, extrapolate_trend="freq"
)
fig = decomp.plot()
fig.set_size_inches(12, 8)
plt.tight_layout()

## Autocorrelation diagnostics

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(series, ax=axes[0], lags=24, title="ACF (24 lags)")
plot_pacf(series, ax=axes[1], lags=24, title="PACF (24 lags)", method="ywm")
plt.tight_layout()

## Monthly profile across years

In [ ]:
monthly_profile = (
    series.to_frame("price")
    .assign(month=series.index.month_name(), month_num=series.index.month)
    .groupby(["month_num", "month"])["price"]
    .agg(["mean", "median", "std"])
    .reset_index()
    .sort_values("month_num")
)
monthly_profile

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(monthly_profile["month"], monthly_profile["mean"], marker="o")
ax.fill_between(
    monthly_profile["month"],
    monthly_profile["mean"] - monthly_profile["std"],
    monthly_profile["mean"] + monthly_profile["std"],
    alpha=0.2,
)
ax.set_ylabel("EUR/MWhe")
ax.set_title("Average seasonal pattern (mean ± std)")
ax.tick_params(axis="x", rotation=45)
plt.tight_layout()

## Heatmap of yearly vs monthly prices

In [ ]:
pivot = (
    series.to_frame("price")
    .assign(year=series.index.year, month=series.index.month_name().str.slice(stop=3))
    .pivot_table(index="year", columns="month", values="price")
    .reindex(
        columns=[
            "Jan",
            "Feb",
            "Mar",
            "Apr",
            "May",
            "Jun",
            "Jul",
            "Aug",
            "Sep",
            "Oct",
            "Nov",
            "Dec",
        ]
    )
)
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(pivot, aspect="auto", cmap="viridis")
ax.set_xticks(range(len(pivot.columns)))
ax.set_xticklabels(pivot.columns)
ax.set_yticks(range(len(pivot.index)))
ax.set_yticklabels(pivot.index)
ax.set_title("Price heatmap (year vs month)")
fig.colorbar(im, ax=ax, label="EUR/MWhe")
plt.tight_layout()

## Takeaways

* The series is monthly and spans the same period as the base dataset; we interpolate small gaps to keep the time-series continuous.
* Seasonal decomposition emphasizes the strong spike during 2021-2022 plus a recurring winter uplift.
* PACF/ACF plots suggest persistence with significant first-order autocorrelation and a seasonal peak around lag 12.
* Use this notebook as a starting point to fit ARIMA/SARIMA or other seasonal models on the aggregated benchmark series (or adapt to individual countries).